<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We use itertools to help with chart colors, matplotlib for plotting, scipy for statistics, yfinance to pull prices, and pykalman for smoothing price series with a Kalman filter.

In [ ]:
from itertools import cycle
import matplotlib.pyplot as plt
import scipy
import yfinance as yf
from pykalman import KalmanFilter

This block sets up a color cycle for future plots so our charts use consistent, readable color choices.

In [ ]:
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])

Here, we download daily closing prices for TSLA and UGA since January 2020 using the yFinance library, then remove any missing values.

In [ ]:
start_date = "2020-01-01"
prices = yf.download(["TSLA", "UGA"], start=start_date).Close.dropna()

This code initializes a Kalman filter. The filter helps us smooth out noise in financial time series so we can see longer-term trends more easily.

In [ ]:
kf = KalmanFilter(
    transition_matrices=[1],
    observation_matrices=[1],
    initial_state_mean=0,
    initial_state_covariance=1,
    observation_covariance=1,
    transition_covariance=0.01,
)

The code above brings in all our tools, prepares them for analysis, and loads up our dataset. The Kalman filter settings allow us to apply a gentle smoothing process to raw price movements for TSLA and UGA. By getting this foundation in place, we can directly analyze and visualize how price trends evolve over time using both raw and smoothed data.

## Visualize smoothed and raw price series

This block applies the Kalman filter to the TSLA price data, plots the original prices as gray points, and plots the smoothed prices as a red line.

In [ ]:
scores_tsla, _ = kf.filter(prices.TSLA)
plt.scatter(prices.index, prices.TSLA, c="gray", s=10, label="TSLA Price")
plt.plot(prices.index, scores_tsla, c="red", label="TSLA MA")
plt.ylabel("TSLA Price")
plt.xlabel("Date")
plt.show()

Here we use the Kalman filter to smooth both TSLA and UGA price series, then plot both moving averages together on the same chart using Matplotlib’s dual y-axis.

In [ ]:
scores_uga, _ = kf.filter(prices.UGA)
scores_tsla, _ = kf.filter(prices.TSLA)

In [ ]:
_, ax1 = plt.subplots()
ax1.plot(prices.index, scores_tsla, color=next(color_cycle), label="TSLA MA")
plt.xlabel("Date")
plt.ylabel("TSLA Price MA")
ax2 = ax1.twinx()
ax2.plot(prices.index, scores_uga, color=next(color_cycle), label="UGA Price MA")
plt.ylabel("UGA Price MA")
plt.show()

In these plots, we see how the Kalman filter helps us reduce day-to-day noise and spot broader movements. The moving average line provides a clean view of what direction prices are generally trending. Using dual axes makes it easy to compare two unrelated securities over time, even if their price levels are very different.

## Measure and visualize statistical relationships

This block defines a function that explores how price movements in the two assets are related over different time lags, using Spearman correlation from the scipy library. It visualizes the correlation at various offsets to spot where the strongest similarities or lead-lag relationships occur.

In [ ]:
def find_offset(ts1, ts2, window):
    l = len(ts1)
    max_i_spearman = -1000
    max_spearman = -1000
    spear_offsets = []

    for i in range(window, 0, -1):
        series1 = ts1[i:]
        series2 = ts2[: l - i]
        spear = scipy.stats.spearmanr(series1, series2)[0]
        spear_offsets.append(spear)

        if spear > max_spearman:
            max_spearman = spear
            max_i_spearman = -i

    for i in range(0, window):
        series1 = ts1[: l - i]
        series2 = ts2[i:]
        spear = scipy.stats.spearmanr(series1, series2)[0]
        spear_offsets.append(spear)
        if spear > max_spearman:
            max_spearman = spear
            max_i_spearman = i

    print("Max Spearman:", max_spearman, " At offset: ", max_i_spearman)
    plt.plot(
        range(-window, window), spear_offsets, c="green", label="Spearman Correlation"
    )
    plt.xlabel("Offset Size (Number of Business Days)")
    plt.ylabel("Spearman Correlation")
    plt.legend(loc=3)
    plt.show()

By running the function below, we examine how smoothed price movements in TSLA and UGA are correlated at different lags and leads, and compare these results to the raw price series.

In [ ]:
print("Kalman-Filtered Smoothed Data")
find_offset(scores_tsla, scores_uga, 200)
print("Raw Data")
find_offset(prices.TSLA, prices.UGA, 150)

This function reveals how the price actions of our selected tickers are statistically connected, even when separated by a number of business days. By scanning across a range of offsets, we highlight where these relationships are most similar, which can point to leadership or lagging effects. Visualizing the correlation results helps us see hidden trends or dependencies between assets beyond what raw plots may show.

## Compare offset moving averages directly

This block lines up the smoothed price series for TSLA and UGA with a specific time offset (here, 140 business days apart) and displays both moving averages on the same chart for easy visual comparison.

In [ ]:
d = 140

In [ ]:
cseries1 = scores_tsla[d:]
cseries2 = scores_uga[: len(scores_tsla) - d]
r = range(len(cseries1))

In [ ]:
_, ax1 = plt.subplots()
ax1.plot(r, cseries1, color=next(color_cycle), label="TSLA MA")
plt.xlabel("Number of Business Days Elapsed")
plt.ylabel("TSLA Price MA")
plt.legend(loc=2)
ax2 = ax1.twinx()
ax2.plot(r, cseries2, color=next(color_cycle), label="UGA Price MA")
plt.ylabel("UGA Price MA")
plt.legend(loc=4)
plt.title(f"{d} Day Offset")
plt.show()

Here we directly compare the trends of both moving averages, visually shifting one series by a significant number of days. This kind of alignment lets us see whether patterns from one asset appear later in the other. Using offset comparisons like this is a hands-on way to uncover possible leading or lagging relationships in how different asset prices move over time.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.